<a href="https://colab.research.google.com/github/mohammad-rahbari/federated-learning_visual-classification/blob/main/notebooks/Federated_learning_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and data

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from torchvision.datasets import CIFAR100
from torch.utils.data import Subset
import numpy as np
import copy
import random
import torch
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
#@title In this block we import the test set of CIFAR100 to evaluate the global model
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=2)

100%|██████████| 169M/169M [00:02<00:00, 78.7MB/s]


# Collecting data of models we want to aggregate

In [4]:
#@title Client's log file has been loaded in this block so we can use it in next steps
clients_data = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv")
clients_data.head()

,client_id,backbone,model_name,initial_model_name,path,num_of_clients,Measurement_criteria,accuracy,loss,train_loss,...,size_of_dataset,client_train_size,client_test_size,train_test_ratio,classes,round_number,duration,time,path_to_subsets,path_to_class_combs
0,44,dino_vits16,c8013d99-5b37-471f-8013-dc0ba691fb8b,28033e92-704c-44bd-abbb-3954b13f00aa,/content/drive/MyDrive/MLDL_FederatedLearning/...,100,"accuracy,loss,train_loss",1.00,6.999795,7.515832,...,50000,400,100,"{'train': 0.8, 'test': 0.2}",all,1,7.914122,2025-08-22 21:16:50,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
1,39,dino_vits16,3884eb9b-feb9-4c34-af8f-048943eedcb4,28033e92-704c-44bd-abbb-3954b13f00aa,/content/drive/MyDrive/MLDL_FederatedLearning/...,100,"accuracy,loss,train_loss",1.75,7.017269,7.684436,...,50000,400,100,"{'train': 0.8, 'test': 0.2}",all,1,7.802925,2025-08-22 21:17:00,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
2,33,dino_vits16,c6d49857-a163-4284-abe3-2bb8bef8615a,28033e92-704c-44bd-abbb-3954b13f00aa,/content/drive/MyDrive/MLDL_FederatedLearning/...,100,"accuracy,loss,train_loss",2.00,6.876796,7.401108,...,50000,400,100,"{'train': 0.8, 'test': 0.2}",all,1,8.174634,2025-08-22 21:17:11,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
3,60,dino_vits16,2c6987b9-5480-4bcb-a48b-d4a9f25b699f,28033e92-704c-44bd-abbb-3954b13f00aa,/content/drive/MyDrive/MLDL_FederatedLearning/...,100,"accuracy,loss,train_loss",0.50,6.773526,7.598617,...,50000,400,100,"{'train': 0.8, 'test': 0.2}",all,1,8.260790,2025-08-22 21:17:21,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
4,63,dino_vits16,344e6d19-9ba8-468a-a564-a8012cec5fb9,28033e92-704c-44bd-abbb-3954b13f00aa,/content/drive/MyDrive/MLDL_FederatedLearning/...,100,"accuracy,loss,train_loss",1.25,6.817880,7.494822,...,50000,400,100,"{'train': 0.8, 'test': 0.2}",all,1,8.120108,2025-08-22 21:17:32,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN


In [7]:
initial_model_name = "28033e92-704c-44bd-abbb-3954b13f00aa" #@param{"type":"string"}


In [8]:
filter =  clients_data['initial_model_name']== initial_model_name
filtered_clients_data = clients_data[filter] # Using filter to collect clients with specified initial model
params = filtered_clients_data[['backbone',
                                    'num_of_clients',
                                    'splitting_method',
                                    'size_of_dataset']]
params = dict(params.iloc[0])
print("Number of all trained clients:", len(clients_data))
print("Number of clients after filtering:", len(filtered_clients_data))
contributors = [] # contributors is being used to store the name of models which contributes in aggregation
for i  in filtered_clients_data['model_name'].values:
  contributors.append(i)
print("Contributors:", contributors)
filtered_clients_data.head()

Number of all trained clients: 10
Number of clients after filtering: 10
Contributors: ['c8013d99-5b37-471f-8013-dc0ba691fb8b', '3884eb9b-feb9-4c34-af8f-048943eedcb4', 'c6d49857-a163-4284-abe3-2bb8bef8615a', '2c6987b9-5480-4bcb-a48b-d4a9f25b699f', '344e6d19-9ba8-468a-a564-a8012cec5fb9', '69309bcf-db48-4b10-85aa-d2c75d035ee0', 'c30cb86b-564a-4b68-9c53-447f81b04606', '5cdd0feb-bdac-4f94-96b8-6edfa263feb6', '73a3d740-3eb7-4252-ae6f-61d1381dff72', '904c260a-f571-4127-ae18-e19448712ed6']


,client_id,backbone,model_name,initial_model_name,path,num_of_clients,Measurement_criteria,accuracy,loss,train_loss,...,size_of_dataset,client_train_size,client_test_size,train_test_ratio,classes,round_number,duration,time,path_to_subsets,path_to_class_combs
0,44,dino_vits16,c8013d99-5b37-471f-8013-dc0ba691fb8b,28033e92-704c-44bd-abbb-3954b13f00aa,/content/drive/MyDrive/MLDL_FederatedLearning/...,100,"accuracy,loss,train_loss",1.00,6.999795,7.515832,...,50000,400,100,"{'train': 0.8, 'test': 0.2}",all,1,7.914122,2025-08-22 21:16:50,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
1,39,dino_vits16,3884eb9b-feb9-4c34-af8f-048943eedcb4,28033e92-704c-44bd-abbb-3954b13f00aa,/content/drive/MyDrive/MLDL_FederatedLearning/...,100,"accuracy,loss,train_loss",1.75,7.017269,7.684436,...,50000,400,100,"{'train': 0.8, 'test': 0.2}",all,1,7.802925,2025-08-22 21:17:00,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
2,33,dino_vits16,c6d49857-a163-4284-abe3-2bb8bef8615a,28033e92-704c-44bd-abbb-3954b13f00aa,/content/drive/MyDrive/MLDL_FederatedLearning/...,100,"accuracy,loss,train_loss",2.00,6.876796,7.401108,...,50000,400,100,"{'train': 0.8, 'test': 0.2}",all,1,8.174634,2025-08-22 21:17:11,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
3,60,dino_vits16,2c6987b9-5480-4bcb-a48b-d4a9f25b699f,28033e92-704c-44bd-abbb-3954b13f00aa,/content/drive/MyDrive/MLDL_FederatedLearning/...,100,"accuracy,loss,train_loss",0.50,6.773526,7.598617,...,50000,400,100,"{'train': 0.8, 'test': 0.2}",all,1,8.260790,2025-08-22 21:17:21,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
4,63,dino_vits16,344e6d19-9ba8-468a-a564-a8012cec5fb9,28033e92-704c-44bd-abbb-3954b13f00aa,/content/drive/MyDrive/MLDL_FederatedLearning/...,100,"accuracy,loss,train_loss",1.25,6.817880,7.494822,...,50000,400,100,"{'train': 0.8, 'test': 0.2}",all,1,8.120108,2025-08-22 21:17:32,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN


In [9]:
#@title Dino Model
import torch
import torch.nn as nn

class DinoClassifire(nn.Module):
  def __init__(self, backbone, num_classes:int=100, device=None):
    super(DinoClassifire, self).__init__()
    self.backbone = torch.hub.load('facebookresearch/dino:main', backbone)

    #We need to freaze thhe parameters of bakbone first so we can train only on the head layer(output layer)
    for param in self.backbone.parameters():
      param.requires_grad = False

    #determine the Device
    if device is None:
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    self.backbone.to(device)

    #To detect the output feature dimontion of backbone we run  Dummy forward pass
    with torch.no_grad():

      dummy_input = torch.randn(1,3,224,224).to(device)
      dummy_out = self.backbone(dummy_input)


      if isinstance(dummy_out, tuple):
        dummy_out = dummy_out[0]
      elif isinstance(dummy_out, dict):
        dummy_out = dummy_out.get("x_norm_clstoken", next(iter(dummy_out.values())))

      #If the output is 3D (B, T, D), we assume first token is the [CLS] token.
      if dummy_out.dim() == 3:
        dummy_feature = dummy_out[:,0]
      else:
        dummy_feature = dummy_out
      feature_dim = dummy_feature.shape[1]
      print("Detected feature dimontion:", feature_dim)


      #Difineing the classification Head
      self.head = nn.Linear(feature_dim, num_classes)

      #Ensure the head is trainable.
      for param in self.head.parameters():
        param.requires_grad = True

  def forward(self,x):

    #pass the input through the backbone
    features = self.backbone(x)

    if isinstance(features, tuple):
      features = features[0]
    elif isinstance(features, dict):
      features = features.get("x_norm_clstoken", next(iter(features.values())))


    # If featers are retuened as (B, T, D), use the first token
    if features.dim() == 3:
      cls_token = features[:,0]
    else:
      cls_token = features
    logits = self.head(cls_token)

    return logits

In [10]:
# @title `get_model` function retrieves and loads the models of filtered clients
def get_model(paths,sample_sizes, backbone):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = DinoClassifire(backbone=backbone, num_classes=100, device=device) # Loading an initial custom dino model
  for index in range(len(paths)):
    state_dict = torch.load(paths.iloc[index]) # load state dict regarding the client number 'index'
    model.head.load_state_dict(state_dict) # set the state dict based on client
    model.to(device)
    yield (model,sample_sizes.iloc[index]) # this command throws model one at the time so less time and resouces will be used






# Aggregation functions

Implemented algorithm:

*   FedAvg
*   FedAvgM
*   EMA



In [11]:
#@title <h2>FedAvg</h2>
def fed_avg(df):
  total_samples = df["client_train_size"].sum() # Calculate the total number of samples of clients wich had contributed
  global_head = None # This variable stores the weights we want to modify

  models = get_model(df["path"],df["client_train_size"], df.iloc[0]["backbone"])
  with torch.no_grad():

    for model, sample_size in models:
      if global_head is None:
        global_head = copy.deepcopy(model.head.state_dict())
        global_model = copy.deepcopy(model)
        for k in global_head.keys():
          global_head[k].zero_() # This command sets the tensor to zero

      for k in global_head.keys():
        global_head[k] += model.head.state_dict()[k] * (sample_size / total_samples) # Each weight will be assgin by average of all clients weights

    global_model.head.load_state_dict(global_head) # A model with modified head will be assignd

  return global_model



In [12]:
#@title FeAvgM
def fed_avg_m(df, momentum_coefficient=0.9, cliping_range=5,  momentum_vector_path= None):

  total_samples = df["client_train_size"].sum() # Calculate the total number of samples of clients wich had contributed
  delta = None # `delta` is variable that keep the average of clients

  models = get_model(df["path"],df["client_train_size"], df.iloc[0]["backbone"])
  with torch.no_grad():

    for model, sample_size in models:
      if delta is None:

        global_model = copy.deepcopy(model)
        global_head = copy.deepcopy(model.head.state_dict())
        delta = { k: torch.zeros_like(v) for k, v in global_head.items() } # A dict with structure of the model that we want to modify will be generated


      client_head = model.head.state_dict()
      for k in delta.keys():

        delta[k] += (client_head[k] - global_head[k]) * (sample_size / total_samples) # Each weight will be assgin by average of all clients weights


    # In this section we calculate the momentum_vector

    if momentum_vector_path is None :
      momentum_vector = copy.deepcopy(delta) # In first round the `delta` as momentum vector
    else:
      momentum_vector = torch.load(momentum_vector_path) #In rounds > 1 momentum vector is requerd to be loaded from drive
      for k in delta.keys():
        momentum_vector[k] = momentum_coefficient * momentum_vector[k]  + delta[k]  # Using the the formula of FedAvgM we calculate the momentum vector







    global_head = copy.deepcopy(global_model.head.state_dict())
    for k in momentum_vector.keys():
      global_head[k] = torch.clamp(global_head[k] + momentum_vector[k], -cliping_range, cliping_range) #After adding momentum vector the last global model we use clamp function we insure to keep momentum vector in boundary

    global_model.head.load_state_dict(global_head)

  return global_model, momentum_vector # We return momentum_vector to save it and use for next aggregation steps



In [13]:
#@title EMA
def ema(df, decay=0.9, cliping_range=5, momentum_vector_path= None):

  total_samples = df["client_train_size"].sum() # Calculate the total number of samples of clients wich had contributed
  delta = None # `delta` is variable that keep the average of clients

  models = get_model(df["path"],df["client_train_size"], df.iloc[0]["backbone"])
  with torch.no_grad():

    for model, sample_size in models:
      if delta is None:

        global_model = copy.deepcopy(model)
        global_head = copy.deepcopy(model.head.state_dict())
        delta = { k: torch.zeros_like(v) for k, v in global_head.items() } # A dict with structure of the model that we want to modify will be generated


      client_head = model.head.state_dict()
      for k in delta.keys():

        delta[k] += (client_head[k] - global_head[k]) * (sample_size / total_samples)  # Each weight will be assgin by average of all clients weights


    # In this section we calculate the momentum_vector
    if momentum_vector_path is None :
      momentum_vector = copy.deepcopy(delta) # In first round the `delta` as momentum vector it is also possible to use aa zero tensor
    else:
      momentum_vector = torch.load(momentum_vector_path)
      for k in delta.keys():
        momentum_vector[k] = decay * momentum_vector[k]  + (1- decay) * delta[k]  # Using the the formula of EMA we calculate the momentum vector


    global_head = copy.deepcopy(global_model.head.state_dict())
    for k in momentum_vector.keys():
      global_head[k] = torch.clamp(global_head[k] + momentum_vector[k], -cliping_range, cliping_range)#After adding momentum vector the last global model we use clamp function we insure to keep momentum vector in boundary

    global_model.head.load_state_dict(global_head)

  return global_model, momentum_vector # We return momentum_vector to save it and use for next aggregation steps



In [14]:
# @title This function will evaluate the model.</br> The outputs are loss and accuracy
def evaluation(model, data_loader):
  criterion = nn.CrossEntropyLoss()
  model.eval()
  correct = 0
  total = 0
  test_loss = 0

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  with torch.no_grad():
    for images, labels in  data_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)

      _, prediction = torch.max(outputs.data,1)
      loss = criterion(outputs, labels)
      test_loss += loss.item() * labels.size(0)

      total += labels.size(0)
      correct += (prediction == labels).sum().item()
    accuracy = 100 * correct / total
    loss = test_loss / total
    return accuracy, loss



In [15]:
# @title <h2>`next_id`</h2> This function generates a unique name for model. `uuid4` does not generate duplicated but we are using a fixed `seed` hence we insure this name does not already exists.

from uuid import uuid4
import os
def next_id(log_path):
  if os.path.exists(log_path):
    df = pd.read_csv(log_path)
    while True:
      uuid = str(uuid4())
      if uuid not in df["model_name"].values:
        return uuid
  else:
    return str(uuid4())

In [16]:
from datetime import datetime

def get_current_time():
  now = datetime.now()

  formatted_date_time = now.strftime("%Y-%m-%d %H:%M:%S") # Format the date and time as a string

  return formatted_date_time

def global_model_name_path_generator():

  model_name = next_id("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")

  path = "/content/drive/MyDrive/MLDL_FederatedLearning/models/global/" + model_name + ".pth"

  return model_name, path



In [17]:
def del_model(model_name):
  log_df = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
  filter = log_df["model_name"] == model_name
  if not filter.any():
    print(f"recored ({model_name}) not found.")
    return
  if os.path.exists(log_df[filter]["path"].values[0]):
    os.remove(log_df[filter]["path"].values[0])
  else:
    print("model not found")


  if not np.isnan(log_df[filter]["momentum_vector_path"].values[0])  and os.path.exists(log_df[filter]["momentum_vector_path"].values[0]):
    os.remove(log_df[filter]["momentum_vector_path"].values[0])
  else:
    print("momentum vector not found")
  log_df = log_df[~filter]
  log_df.to_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv", index=False)
# del_model("54ba3a96-df4d-4e55-bea1-fa8547aff906")

In [ ]:
# #@title use this block to modify the global log file
# temp = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
# temp["aggregation_method"] =[ np.nan] * len(temp)
# temp["contributors"] =[ np.nan] * len(temp)
# temp["momentum_vector_path"] = [np.nan] * len(temp)

# temp = temp[['backbone',
#               'num_of_clients',
#               'splitting_method',
#               'aggregation_method',
#               'Measurement_criteria',
#               'accuracy',
#               'loss',
#               'size_of_dataset',
#               'train_test_ratio',
#               'classes',
#               'round_number',
#               'num_of_participants',
#               'model_name',
#               'prev_global_model_name',
#               "contributors",
#               'path',
#               "momentum_vector_path",
#               'path_to_subsets',
#               'path_to_class_combs',
#               'time'
#                ]]
# temp.to_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv", index=False)
# temp.head()
# del temp

In [18]:
#@title <h1>Select aggregation method </h1>
#@markdown This value will be auto assigned in case the initial model of filtered clients have been aggregated with a spicific aggregation function in the previous rounds

aggregation_method = "FedAvg"   #@param["FedAvg","FedAvgM", "EMA" ]

prev_agg_method = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
filter = prev_agg_method["model_name"] == initial_model_name
prev_agg_method = prev_agg_method[filter]

prev_agg_method = prev_agg_method["aggregation_method"].values[0]
aggregation_method = aggregation_method if np.isnan( prev_agg_method) else prev_agg_method


print(f"{aggregation_method} has been selected as the Aggregation function.")

FebAvg has been selected as the Aggregation function.


In [19]:
if aggregation_method == "FedAvg":
  global_model = fed_avg(filtered_clients_data)
elif aggregation_method == "FedAvgM":
  global_model,momentum_vector = fed_avg_m(filtered_clients_data)
elif aggregation_method == "EMA":
  global_model,momentum_vector = ema(filtered_clients_data)



model_name, path = global_model_name_path_generator()

test_accracy, test_loss= evaluation(global_model, test_loader)
print(f"Accurace:{test_accracy:.2f}")
print(f"Loss:{test_loss:.2f}")

Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_deitsmall16_pretrain.pth


100%|██████████| 82.7M/82.7M [00:00<00:00, 228MB/s]


Detected feature dimontion: 384
Accurace:1.24
Loss:6.97


In [20]:
import os
log_path = "/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv"

prev_global_model_name = filtered_clients_data["initial_model_name"].values[0]
global_model_log = filtered_clients_data.drop(["client_id","train_loss","client_train_size","client_test_size","duration",],axis=1)
global_model_log = global_model_log.iloc[0]
global_model_log["num_of_participants"] = len(filtered_clients_data)
global_model_log["prev_global_model_name"] = initial_model_name
global_model_log["model_name"]= model_name
global_model_log["accuracy"] = test_accracy
global_model_log["loss"] = test_loss
global_model_log["time"] = get_current_time()
global_model_log["path"] = path
global_model_log["Measurement_criteria"] = "accuracy,loss"
global_model_log["contributors"] = contributors
global_model_log["aggregation_method"] = aggregation_method

if aggregation_method == "FedAvg":
  global_model_log["momentum_vector_path"] = None

elif aggregation_method == "FedAvgM" or aggregation_method == "EMA":
  global_model_log["momentum_vector_path"] = "/content/drive/MyDrive/MLDL_FederatedLearning/models/global/momentun_vectors/MV_"+ model_name + ".pt"
  torch.save(momentum_vector, global_model_log["momentum_vector_path"])


global_model_log = pd.DataFrame(global_model_log).T
global_model_log = global_model_log[['backbone',
              'num_of_clients',
              'splitting_method',
              'aggregation_method',
              'Measurement_criteria',
              'accuracy',
              'loss',
              'size_of_dataset',
              'train_test_ratio',
              'classes',
              'round_number',
              'num_of_participants',
              'model_name',
              'prev_global_model_name',
              "contributors",
              'path',
              "momentum_vector_path",
              'path_to_subsets',
              'path_to_class_combs',
              'time'
               ]]

flag = False
if os.path.exists(path):
  print("Model already exist")
  flag = True

if os.path.exists(log_path):
  global_log_df = pd.read_csv(log_path)
  if model_name in global_log_df["model_name"].values:
    print("record already exist")
    flag = True
  if not flag:
    global_model_log.to_csv(log_path, mode='a', header=False, index=False)
else:
  if not flag:
    global_model_log.to_csv(log_path, index=False, header=True)


if not flag:
  torch.save(global_model.state_dict(),  path )
  if aggregation_method == "FedAvgM" or aggregation_method == "EMA":
    torch.save(momentum_vector, global_model_log["momentum_vector_path"].values[0])


In [21]:
global_model_log.head()

,backbone,num_of_clients,splitting_method,aggregation_method,Measurement_criteria,accuracy,loss,size_of_dataset,train_test_ratio,classes,round_number,num_of_participants,model_name,prev_global_model_name,contributors,path,momentum_vector_path,path_to_subsets,path_to_class_combs,time
0,dino_vits16,100,i.i.d. sharing,FebAvg,"accuracy,loss",1.24,6.967797,50000,"{'train': 0.8, 'test': 0.2}",all,1,10,d879f2d8-a17b-4a22-8957-338d9a9f292c,28033e92-704c-44bd-abbb-3954b13f00aa,"[c8013d99-5b37-471f-8013-dc0ba691fb8b, 3884eb9...",/content/drive/MyDrive/MLDL_FederatedLearning/...,None,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN,2025-08-22 21:26:38
